In [3]:
!unset DYLD_FALLBACK_LIBRARY_PATH

In [4]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import osmnx
import igraph as ig
from spopt.locate import PMedian
import pulp
import networkx as nx
import random
import warnings
from tqdm import auto as tqdm

SMALL_SIZE = 5
matplotlib.rc('font', size=SMALL_SIZE)
matplotlib.rc('axes', titlesize=SMALL_SIZE)

random.seed(1234)

COLORS = {"S": "#2f4b7c",
          "E": "#ffa600",
          "I": "#f95d6a",
          "R": "#a05195",
          "D": "#003f5c",
          "C": "#ff7c43",
          "X": "#665191",
          "Y": "#d45087"}

warnings.filterwarnings("ignore")-----------------------------------------------
OSError                                   Traceback (most recent call last)
<ipython-input-4-b525cfff

OSError: Could not find lib c or load any of its variants [].

In [2]:
!pip uninstall shapely

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


In [ ]:
%load_ext watermark
%watermark --iversions -v -a "Chris von Csefalvay" -gu "chrisvoncsefalvay" -p pysal,cplex,spopt

In [ ]:
location = "Alexandria, VA"
p = 4

In [ ]:
%%time
graph = osmnx.graph_from_place(location, network_type="drive")

In [ ]:
osmnx.plot_graph(graph)

In [ ]:
graph = osmnx.distance.add_edge_lengths(graph)

In [ ]:
osm_ids = list(graph.nodes)
graph = nx.relabel.convert_node_labels_to_integers(graph)

osm_id_vals = {k: v for k, v in zip(graph.nodes, osm_ids)}
nx.set_node_attributes(graph, osm_id_vals, "osmid")

i_graph = ig.Graph(directed=True)
i_graph.add_vertices(graph.nodes)
i_graph.add_edges(graph.edges())
i_graph.vs["osmid"] = osm_ids
i_graph.es["length"] = list(nx.get_edge_attributes(graph, "length").values())

In [ ]:
building_filter = ["yes", "dormitory", "terrace", "apartments", "residential", "house", "semidetached_house", "detached", "condominium", ]

In [ ]:
%%time
buildings = osmnx.geometries.geometries_from_place(location, tags={"building": True})[["geometry", "building"]]
buildings = buildings[buildings.building.isin(building_filter)]
buildings.geometry = buildings.geometry.to_crs(6350)
buildings = buildings[buildings.geometry.apply(lambda x: x.type != "Point")]

In [ ]:
buildings["nearest_node"] = osmnx.distance.nearest_nodes(graph, buildings.geometry.centroid.x, buildings.geometry.centroid.y)
buildings = buildings.reset_index()

In [ ]:
len(buildings)

In [ ]:
osmnx.plot_footprints(buildings)

In [ ]:
%%time
eligible_sites = osmnx.geometries.geometries_from_place(location, tags={"amenity": ["hospital", "community_centre", "parking", "conference_centre", "events_venue"]})[["geometry"]]
eligible_sites.geometry = eligible_sites.geometry.to_crs(6350)
eligible_sites["nearest_node"] = osmnx.distance.nearest_nodes(graph, eligible_sites.geometry.centroid.x, eligible_sites.geometry.centroid.y)
eligible_sites = eligible_sites[eligible_sites.geometry.area > 10000]
eligible_sites = eligible_sites[eligible_sites.geometry.apply(lambda x: x.type != "Point")]
eligible_sites = eligible_sites.reset_index()

In [ ]:
len(eligible_sites)

In [ ]:
osmnx.plot_footprints(eligible_sites)

In [ ]:
cost_matrix = np.zeros(shape=(len(buildings), len(eligible_sites)), dtype=float)

In [ ]:
def get_distance(source_node, destination_node) -> float:
   return sum(list(map(lambda x: i_graph.es[x]["length"], i_graph.get_shortest_paths(v=source_node, to=destination_node, weights="length", output="epath")[0])))

In [ ]:
%%time

for i in tqdm.notebook_trange(len(buildings)):
    for j in range(len(eligible_sites)):
        cost_matrix[i, j] = get_distance(buildings.loc[i, "nearest_node"], eligible_sites.loc[j, "nearest_node"])
        cost_matrix_idx = buildings.index[i]

In [ ]:
cost_df = pd.DataFrame(cost_matrix, index=buildings.index)
cost_df = cost_df.loc[(cost_df != 0).any(axis=1)]

In [ ]:
buildings = buildings.iloc[cost_df.index.to_list()]

In [ ]:
buildings

In [ ]:
cost_matrix = cost_df.to_numpy()

In [ ]:
cost_matrix

In [ ]:
pmedian = PMedian.from_cost_matrix(cost_matrix, weights=np.ones(cost_matrix.shape[0]), p_facilities=p)

In [ ]:
%%time

pmedian.solve(pulp.PULP_CBC_CMD())

In [ ]:
pmedian.facility_client_array()
pmedian.client_facility_array()

In [ ]:
selected_facilities = np.unique(pmedian.cli2fac).tolist()

In [ ]:
selected_facilities

In [ ]:
pmedian.cli2fac

In [ ]:
buildings["assigned_facility"] = list(map(lambda x: x[0], pmedian.cli2fac))

In [ ]:
conversion = dict(zip(buildings.assigned_facility.unique(), list("ABCDEFGHIJKLMNOPQ")[:len(buildings.assigned_facility.unique())]))

In [ ]:
buildings["assigned_facility_label"] = buildings.assigned_facility.apply(lambda x: conversion[x])

In [ ]:
selected_sites = eligible_sites.iloc[selected_facilities]
selected_sites["label"] = selected_sites.index.map(lambda x: conversion[x])

buildings["assigned_facility_distance"] = buildings.apply(lambda x: get_distance(x.nearest_node, eligible_sites.iloc[x.assigned_facility].nearest_node), axis=1)

In [ ]:
selected_sites

In [ ]:
nodes, edges = osmnx.graph_to_gdfs(graph)
edges.geometry = edges.geometry.to_crs(4326)
selected_sites.geometry = selected_sites.geometry.to_crs(4326)
buildings.geometry = buildings.geometry.to_crs(4326)
eligible_sites.geometry = eligible_sites.geometry.to_crs(4326)

circle_centers = selected_sites.geometry.centroid
circle_centers.reset_index()
circle_size=150
middle_scale = "viridis"
bottom_scale = "inferno_r"

fig = plt.figure(facecolor="w", figsize=(9, 10), dpi=600)
ax = fig.add_subplot(211, axisbelow=True)

arterial_roads = edges.loc[edges.highway.isin(["motorway", "trunk", "primary", "primary_link"])]
secondary_roads = edges.loc[edges.highway.isin(["secondary", "tertiary", "residential"])]

arterial_roads.plot(ax=ax, lw=0.5, color='gray', zorder=3)
secondary_roads.plot(ax=ax, lw=0.25, color='gray', zorder=3)

buildings[buildings.assigned_facility_distance > 0].plot("assigned_facility_label", cmap=middle_scale, ax=ax, alpha=0.8)

eligible_sites.plot(ax=ax, color="k", alpha=0.15, zorder=4)

selected_sites.plot("label", ax=ax, cmap=middle_scale, zorder=7, legend=False)

ax.scatter(circle_centers.x, circle_centers.y, marker="o", edgecolors="k", facecolors="none", s=circle_size, zorder=7)

ax = fig.add_subplot(212, axisbelow=True)

arterial_roads = edges.loc[edges.highway.isin(["motorway", "trunk", "primary", "primary_link"])]
secondary_roads = edges.loc[edges.highway.isin(["secondary", "tertiary", "residential"])]

arterial_roads.plot(ax=ax, lw=0.5, color='gray', zorder=3)
secondary_roads.plot(ax=ax, lw=0.25, color='gray', zorder=3)

eligible_sites.plot(ax=ax, color="k", alpha=0.2, zorder=4)

selected_sites.plot(ax=ax, color=COLORS["X"], zorder=5)

ax.scatter(circle_centers.x, circle_centers.y, marker="o", edgecolors="k", facecolors="none", s=circle_size, zorder=7)

dist = buildings[buildings.assigned_facility_distance > 0].plot("assigned_facility_distance", ax=ax, cmap=bottom_scale, legend=False)

norm = colors.Normalize(vmin=buildings.assigned_facility_distance.min(), vmax=buildings.assigned_facility_distance.max())
cbar = plt.cm.ScalarMappable(norm=norm, cmap=bottom_scale)


axins = inset_axes(ax,
                    width="40%",  # width = 50% of parent_bbox width
                    height="3%",  # height : 5%
                    loc='upper left',
                   borderpad=3)



ax_cbar = fig.colorbar(cbar, cax=axins, orientation="horizontal")

plt.savefig("testing_site_loc_opt.pdf")